# 1. Install, Imports, Settings

In [49]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import spacy
from spacy.lang.es.examples import sentences 
#!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")
from collections import Counter
import string, math, json

In [50]:
plt.rcParams["figure.figsize"] = [10, 6]
%config InlineBackend.figure_format = 'retina'

punct = string.punctuation +'”“'

# 2. Loading data

In [51]:
grupos = {
    "Grupo Armado Organizado" : ["GAO"],
    "Grupo Armado Ilegal" : ["GAI"],
    "Grupo Armado Organizado Residual" : ["GAOR", "GAO-R", "grupo armado organizado - residual" ], 
    "Ejército de Liberación Nacional" : ["ELN"],
    "Autodefensas Gaitanistas de Colombia" : ["AGC"],
    "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" : ["FARC", 
                                         "Fuerzas Armadas Revolucionarias de Colombia",
                                         "Fuerzas Armadas Revolucionarias Ejército del Pueblo",
                                         "FARCEP", "FARC-EP", "FARC EP"
                                        ],
    "Grupo delincuencial organizado" : ["GDO", ],
    "Jalisco Nueva Generación": [],
    "Bandas Criminales" : ["BACRIM", "Bandas Emergentes"],
    "Autodefensas Unidas de Colombia" : ["AUC", ],
    "Grupos posdesmovilización": [],
    "Ejército de Liberación Popular": ["EPL"],
    "Clan del Golfo" : ["AGC"],
}

children = ["NNAJ", "niños", "niñas", "adolescentes", "jóvenes", "NNA"]

narco_grupos  = ["Cartel del Amazonas",  
        "Cartel de Los Nevados",
        "Cartel de Bogotá",
        "Cartel de Buga",
        "Cartel de Medellín",
        "Oficina de Envigado",
        "Cartel de Cali",
        "Cartel del Caquetá",
        "Cartel del Norte del Valle",
        "Los Rastrojos",
        "Cartel de la Costa Atlántica",
        "Cartel de La Guajira",
        "Cartel del Cauca",
        "Los Caparrapos",
        "Los Paisas",
        "Águilas Negras",
        "Los Puntilleros",
        "Ejército Revolucionario Popular Antisubversivo de Colombia",
        "Los Pelusos",
        "GDO Los Pachenca",
        "Clan Gnecco Cerchar",
        "Clan Bustamante", 
        "Activo Primer Frente",
        "Segunda Marquetalia", 
        "Los Zetas", 
        "Cartel de Sinaloa", 
        "Oficina de Envigado", 
        "La Terraza",
        "Robledo",
        "Pachelly",
        "La Sierra", 
        "Caicedo", 
        "Los Triana",
        "Clan del Oriente", 
        "La Unión",
        "Clan del Norte",
        "Los Costeños", 
        "Los Pachencha", 
        "La Cordillera",
        "La Gran Alianza",
        "Banda Local",
        "Norte del Valle",
        "Los Chuckys",
        "Los Chacales",
        "La Constru",
        "Los Caqueteños"]

POI = {
    "Evaristo Porras": "Papa Doc",
    "Luis Agustín Caicedo Velandia" : "Don Lucho", 
    "Víctor Manuel Mejía Múnera": "Pablo Arauca",
    "Miguel Ángel Mejía Múnera": "El Mellizo",
    "Ramón Quintero":"",
    "Pablo Escobar" : "El patrón",
    "Gonzalo Rodríguez Gacha": "",
    "Diego Fernando Murillo": "",
    "Gilberto Rodríguez Orejuela": "El Ajedrecista",
    "Miguel Rodríguez Orejuela": "",
    "Leonidas Vargas": "El Rey de Caquetá",
    "Luis Enrique Calle Serna": "",
    "Wilber Varela": "",
    "Alberto Orlandez Gamboa": "", 
    "Hermágoras González": "",
    "Nicolás Rodríguez Bautista" : " Gabino ", 
    "Iván Márquez": "",
    "Gentil Duarte": "",
    "Jorge Gnecco Cerchar": "", 
    "Jesús María Aguirre" :"Chucho Mercancía",
    "Megateo": "",  
    "Vicente Castaño": "", 
    "Emiliano Alcides Osorio": " Caín ",
    "Edgar Bustamante" : "El Yuca",
    "Manuel Marulanda Vélez" :" Tirofijo ",     
    "Jorge Briceño Suarez Mono": " Jojoy ",
    "Dairo Antonio Úsuga David": "",
    "Salvatore Mancuso": "",
    "Rodrigo Tovar": "",
    "Carlos Castaño Gil": "",
    "Justo Pastor Perafán": "Don Pepe"
}

loc = pd.read_csv("resources/Departamentos_y_municipios_de_Colombia.csv")
loc.head()

,REGION,CÓDIGO DANE DEL DEPARTAMENTO,DEPARTAMENTO,CÓDIGO DANE DEL MUNICIPIO,MUNICIPIO
0,Región Eje Cafetero - Antioquia,5,Antioquia,5.001,Medellín
1,Región Eje Cafetero - Antioquia,5,Antioquia,5.002,Abejorral
2,Región Eje Cafetero - Antioquia,5,Antioquia,5.004,Abriaquí
3,Región Eje Cafetero - Antioquia,5,Antioquia,5.021,Alejandría
4,Región Eje Cafetero - Antioquia,5,Antioquia,5.030,Amagá


In [52]:
dfnnaj = pd.read_csv("data/alertas_actors_NNAJ.csv", sep="|")
df = pd.read_csv("data/alertas_actors.csv", sep="|")
print("Total Number of documents:", len(df))
print("Total Number of documents NNAJ:", len(dfnnaj)) 

Total Number of documents: 1752
Total Number of documents NNAJ: 1055


In [53]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Filename,Subtype,Type,Year,Path,Departamento,Actors
0,0,0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,1,1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,2,2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,3,3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,4,4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


In [54]:
dfnnaj.tail(5)

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text
1050,IS N° 002-2022 a AT N° 079-18 VAL-Buenaventura...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 002-20...,Valle del Cauca,['Entidad Respuesta Institucional Ministerio d...,Entidad Respuesta Institucional Ministerio de ...
1051,IS N° 003-2022 a AT N° 034-20 NSANT-Ocaña.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 003-20...,Norte de Santander,"['Instituto Colombiano de Bienestar Familiar',...","Alcaldía de Ocaña A la alcaldía de Ocaña, para..."
1052,IS N° 006-2022 a AT N° 004-2021.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 006-20...,Norte de Santander,"['La Constru', 'La Constru', 'Servicio Naciona...","En su conjunto, y advirtiendo que los grupos a..."
1053,IS N° 011-2022 a AT N° 031-2020 MET-San Martín...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 011-20...,Meta,"['Autodefensas Gaitanistas de Colombia', 'Band...",Respecto del escenario de riesgo descrito en l...
1054,IS N° 012-2022 a AT N° 042 de 2020 BOY-El Cocu...,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 012-20...,Boyacá,"['Grupo Armado Ilegal', 'La Sierra', 'La Sierr...",Es así como en el municipio de Chita en el mes...


# 3. Cleaning Duty

In [55]:
arts = ['la', 'los', 'el', 'las', 'una', 'un']
def replace_accents(w):
    return w.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')

arts = ['la', 'los', 'el', 'las', 'una', 'un']
def replace_articles(w):
    w = [x for x in w.split() if x not in arts]
    return ' '.join(w)

def clean_segment(s):
    return replace_articles(replace_accents(s.lower()))
    
def clean_actors(entities):
    entities = entities.strip("']['").split("', '")
   
    if len(entities) == 0:
        return []
    
    #grupos
    for key, value in grupos.items():
        for v in value:
            entities = [key if clean_segment(e) == clean_segment(v) else e for e in entities]
           
    #narco grupos
    for v in narco_grupos:
        entities = [v if clean_segment(e) == clean_segment(v) else e for e in entities]
    return entities

df["Actors"] = df["Actors"].apply(clean_actors)
dfnnaj["Actors_NNAJ"] = dfnnaj["Actors_NNAJ"].apply(clean_actors)

In [56]:
len(df), len(dfnnaj)

(1752, 1055)

In [57]:
df.to_csv("data/alertas_actors.csv", sep="|")
df = df.set_index('Filename')
#remove duplicates
df = df[~df.index.duplicated(keep='first')]
df = df
df.head()

,Unnamed: 0.1,Unnamed: 0,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,,,
"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Policarpa y Roberto Payán.pdf",0,0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Aut..."
AT N° 004-18 NAR-Tumaco.pdf,1,1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Gru..."
AT N° 005-18 COR-Tierralta.pdf,2,2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Ejé..."
AT N° 006-18 ARA-Saravena.pdf,3,3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Ejé..."
"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y Vista Hermosa.pdf",4,4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"[Grupo Armado Organizado, Grupo Armado Organiz..."


In [58]:
dfnnaj.to_csv("data/alertas_actors_NNAJ.csv", sep="|")
dfnnaj = dfnnaj.set_index('Filename')
dfnnaj.tail()

,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text
Filename,,,,,,,
IS N° 002-2022 a AT N° 079-18 VAL-Buenaventura.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 002-20...,Valle del Cauca,[Entidad Respuesta Institucional Ministerio de...,Entidad Respuesta Institucional Ministerio de ...
IS N° 003-2022 a AT N° 034-20 NSANT-Ocaña.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 003-20...,Norte de Santander,"[Instituto Colombiano de Bienestar Familiar, I...","Alcaldía de Ocaña A la alcaldía de Ocaña, para..."
IS N° 006-2022 a AT N° 004-2021.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 006-20...,Norte de Santander,"[La Constru, La Constru, Servicio Nacional de ...","En su conjunto, y advirtiendo que los grupos a..."
IS N° 011-2022 a AT N° 031-2020 MET-San Martín.pdf,Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 011-20...,Meta,"[Autodefensas Gaitanistas de Colombia, Bandas ...",Respecto del escenario de riesgo descrito en l...
"IS N° 012-2022 a AT N° 042 de 2020 BOY-El Cocuy, Güicán de la Sierra, Panqueba, El Espino, Chiscas, Chita, Jericó, Socha y Socotá.pdf",Informe de Seguimiento,Seguimiento,2022,data\Informes de Seguimiento\2022\IS N° 012-20...,Boyacá,"[Grupo Armado Ilegal, La Sierra, La Sierra, La...",Es así como en el municipio de Chita en el mes...


In [60]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
df.head()

,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,
"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Policarpa y Roberto Payán.pdf",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Aut..."
AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Gru..."
AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Ejé..."
AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"[Grupo Armado Ilegal, Grupo Armado Ilegal, Ejé..."
"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y Vista Hermosa.pdf",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"[Grupo Armado Organizado, Grupo Armado Organiz..."


In [61]:
#merging DFs
dftotal = pd.concat([dfnnaj, df.reindex(dfnnaj.index)], axis=1)
len(dftotal)

1055

In [62]:
dftotal[dftotal['Actors_NNAJ'].isnull()]

,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text,Subtype,Type,Year,Path,Departamento,Actors
Filename,,,,,,,,,,,,,


In [69]:
dfnnaj = dfnnaj.reset_index()
dfnnaj.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,[],Por lo anterior; desde el Sistema de Alertas T...
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"[Grupo Armado Ilegal, Fuerzas Armadas Revoluci...","Los territorios dejados por las FARC - EP, fue..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,[Grupo Armado Ilegal],"En ese sentido, la población civil se encuentr..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"[Derecho Internacional Humanitario, Policía]","También se advierte· el riesgo de secuestros, ..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"[Nuev, Partido FAR, Defensoria del Pueblo, Fue...",Integrantes del Nuev o Partido FAR( **niños** ...


In [70]:
len(dfnnaj)

1055

In [71]:
dfnnaj.to_csv("data/alertas_actors_NNAJ.csv", sep="|", index=False)

# 4. Functions

In [72]:
df = pd.read_csv("data/alertas_actors_NNAJ.csv", sep="|")
df.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,[''],Por lo anterior; desde el Sistema de Alertas T...
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Fuerzas Armadas Revol...","Los territorios dejados por las FARC - EP, fue..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,['Grupo Armado Ilegal'],"En ese sentido, la población civil se encuentr..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Derecho Internacional Humanitario', 'Policía']","También se advierte· el riesgo de secuestros, ..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Nuev', 'Partido FAR', 'Defensoria del Pueblo...",Integrantes del Nuev o Partido FAR( **niños** ...


In [73]:
space_articles = []
red_flags = ['Telefax', 'procesoSubproceso Prevención y Protección Código','SAT', 'Alerta Temprana', 
             'Derecho Internacional Humanitario', 'SAT Código DPR', 
             'procesoSubproceso Prevención y Protección Código PPP02F10 ALERTA', 'SEGUIMIENTO', 
             'PARA', 'Frente', 'Nota de Seguimiento', 'PP', 'AT', 'LIBERTAD',  'Alertas Tempranas', 
             'P01 F07 Formato Nota de Seguimiento Versión 03 Vigente', 
             'Prevención y Protección Código PP', 'Bloque C', 'Comisión', 'POBLACION','Constitución Política', 
             'procesoSubproceso Prevención y Protección Código PP', 'Ejército', 'Sin', 'INFORME', 'MUSE', 
             'BIENES INDISPENSABLES PARA', 'Gobernación', 'A NO SER', 'cia', 'Alerta', 'RECOMENDACIONES A', 
            'NNA', 'Adolescentes', 'ADOLESCENTES', 'SAT INFORME DE RIESGO No']

arts = ['la', 'los', 'el', 'las']

In [74]:
def remove_flags(entities):
     return [e for e in entities if e not in red_flags]

def remove_articles(entities):
    new = []
    for i in range(0, len(entities) - 1):
            new.append(entities[i])
            for a in arts:
                if entities[i].lower().startswith(a + " "):
                    new[len(new)-1] = entities[i][len(a):]

    return new
                     
def fix_terms(entities):
    new = []
    for i in range(0, len(entities)):
        new.append(entities[i])
        # fix ELN
        if entities[i].startswith("ELN "):
                   new[len(new) - 1] =  "Ejército de Liberación Nacional"
        #fix Las FARC           
        elif entities[i].startswith("FARC") or entities[i].startswith("lasFARC") or entities[i].startswith("LasFARC"):
                   new[len(new) - 1] = "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" 
                
        elif entities[i].startswith("Fuerzas Armadas Revolucionarias"):
                   new[len(new) - 1] = "Fuerzas Armadas Revolucionarias - Ejército del Pueblo" 
                
        elif entities[i].lower().startswith("urabe"):
                   new[len(new) - 1] = "Urabeños"     
                
        elif entities[i].startswith("UNP"):
                   new[len(new) - 1] = "Unidad Nacional de Protección"    
        
        elif entities[i].startswith("Acción Social"):
                   new[len(new) - 1] = "Agencia Presidencial para la Acción Social"
                
        elif entities[i].startswith("Al Instituto Colombiano de Bienestar Familiar") or entities[i].startswith("ICBF"):
                   new[len(new) - 1] = "Instituto Colombiano de Bienestar Familiar"
                
        elif entities[i].lower().startswith("armada"):
                   new[len(new) - 1] = "Armada Nacional"
                
        elif entities[i].lower().startswith("defensor"):
                   new[len(new) - 1] = "Defensoría del Pueblo"
                
    return new

def clean_list_actors(df, column):
    actors = df[column].sum()
    if type(actors) == str:
        actors = actors.strip("']['").split("', '")
    
    #remove articles
    actors = remove_articles(actors)
                     
    #fix some terms
    actors = fix_terms(actors)                               
                
    #remove red flags
    actors = remove_flags(actors)
    
    #remove blank spaces
    actors = [a.strip() for a in actors]
    
    return actors
    
def most_mentioned_actors(actors):           
    data = { 
             'actors': list(Counter(actors).keys()), 
             'frequency': list(Counter(actors).values())
           }

    counterdf = pd.DataFrame(data)
    counterdf = counterdf.groupby('actors').agg({'frequency' : 'sum'}).sort_values('frequency', ascending = False)
    return counterdf

# 4. Most Mentioned Actors - Top 20 in Documents that mention NNAJ

#### Los articulos se omiten por cuestiones de estandarizacion de los datos, pero por ejemplo Rastrojos se refiere a Los Rastrojos que es el nombre original, as'i pasa con Los Paisas, La Cordillera, La Constru, etc. 

In [29]:
dfa = most_mentioned_actors(clean_list_actors(df, 'Actors'))
dfa[:20]

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,11224
Ejército de Liberación Nacional,5755
Autodefensas Unidas de Colombia,4965
Fuerza Pública,4289
Águilas Negras,3196
Rastrojos,2556
Policía Nacional,2410
Autodefensas Gaitanistas de Colombia,2067
Grupo Armado Ilegal,1655


# 4.1 Most Mentioned Actors - Top 20 in the Same Sentence that mention  NNAJ - More Context

In [30]:
dfb = most_mentioned_actors(clean_list_actors(df, 'Actors_NNAJ'))
dfb[:20]

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,528
Ejército de Liberación Nacional,254
Autodefensas Unidas de Colombia,223
Águilas Negras,158
Instituto Colombiano de Bienestar Familiar,133
Rastrojos,127
Fuerza Pública,126
Urabeños,106
Autodefensas Gaitanistas de Colombia,78


In [31]:
df.head()

,Filename,Subtype,Type,Year,Path,Departamento,Actors_NNAJ,NNAJ_Text,Unnamed: 0,Subtype.1,Type.1,Year.1,Path.1,Departamento.1,Actors
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,[''],Por lo anterior; desde el Sistema de Alertas T...,0,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
1,AT N° 004-18 NAR-Tumaco.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Fuerzas Armadas Revol...","Los territorios dejados por las FARC - EP, fue...",1,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
2,AT N° 005-18 COR-Tierralta.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,['Grupo Armado Ilegal'],"En ese sentido, la población civil se encuentr...",2,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
3,AT N° 006-18 ARA-Saravena.pdf,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Derecho Internacional Humanitario', 'Policía']","También se advierte· el riesgo de secuestros, ...",3,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca,"['Grupo Armado Ilegal', 'Grupo Armado Ilegal',..."
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Nuev', 'Partido FAR', 'Defensoria del Pueblo...",Integrantes del Nuev o Partido FAR( **niños** ...,4,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta,"['Grupo Armado Organizado', 'Grupo Armado Orga..."


# 5. Statistical Distribution of Actors

In [32]:
dfa[dfa.frequency > 100].describe()

,frequency
count,104.000000
mean,631.596154
std,1415.978364
min,101.000000
25%,124.000000
50%,169.000000
75%,474.500000
max,11224.000000


# 6. Top 10 actors in each period

In [33]:
df1 = df[df.Year < 2006]
df2 = df[(df.Year > 2005) & (df.Year < 2012)]
df3 = df[(df.Year > 2011) & (df.Year < 2017)]
df4 = df[df.Year > 2016]

# 6.1 2001 - 2005 (Conflicto, 2005 Justicia y Paz)

In [34]:
df1 = most_mentioned_actors(clean_list_actors(df1, 'Actors'))[:10]
df1

,frequency
actors,
Autodefensas Unidas de Colombia,855
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,843
Ejército de Liberación Nacional,180
Fuerza Pública,162
Autodefensas,150
Red de Solidaridad Social,90
Armada Nacional,82
Cordillera,60
OEA,47


In [35]:
df1[df1.frequency > 100].describe()

,frequency
count,5.000000
mean,438.000000
std,375.365822
min,150.000000
25%,162.000000
50%,180.000000
75%,843.000000
max,855.000000


# 6.2 2006- 2011 (Conflicto)

In [36]:
df2 = most_mentioned_actors(clean_list_actors(df2, 'Actors'))[:10]
df2

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,2871
Autodefensas Unidas de Colombia,1684
Águilas Negras,1542
Fuerza Pública,1138
Rastrojos,964
Ejército de Liberación Nacional,790
Agencia Presidencial para la Acción Social,655
Grupo Armado Ilegal,410
Paisas,385


In [37]:
df2[df2.frequency > 100].describe()

,frequency
count,10.000000
mean,1081.600000
std,781.280601
min,377.000000
25%,471.250000
50%,877.000000
75%,1441.000000
max,2871.000000


# 6.3 2012 - 2016 (Negociaciones de Paz, Acuerdo de La Habana)

In [38]:
df3 = most_mentioned_actors(clean_list_actors(df3, 'Actors'))[:10]
df3

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,3179
Ejército de Liberación Nacional,1246
Fuerza Pública,1245
Rastrojos,1224
Autodefensas Unidas de Colombia,1066
Urabeños,1055
Águilas Negras,1049
Autodefensas Gaitanistas de Colombia,725
Policía Nacional,666


In [39]:
df3[df3.frequency > 100].describe()

,frequency
count,10.000000
mean,1201.200000
std,738.403353
min,557.000000
25%,806.000000
50%,1060.500000
75%,1239.750000
max,3179.000000


# 6.4 2017 - 2022 (Post- Acuerdo)

In [40]:
df4 = most_mentioned_actors(clean_list_actors(df4, 'Actors'))[:10]
df4

,frequency
actors,
Fuerzas Armadas Revolucionarias - Ejército del Pueblo,4331
Ejército de Liberación Nacional,3539
Fuerza Pública,1744
Autodefensas Unidas de Colombia,1360
Policía Nacional,1338
Autodefensas Gaitanistas de Colombia,1148
Constru,946
Unidad Nacional de Protección,912
Grupo Armado Ilegal,683


In [41]:
df4[df4.frequency > 100].describe()

,frequency
count,10.000000
mean,1660.600000
std,1258.950904
min,605.000000
25%,920.500000
50%,1243.000000
75%,1648.000000
max,4331.000000


# 6. Export HTML

In [42]:
!jupyter nbconvert --to html 14_Mapping_Actores_del_Conflicto_NNAJ.ipynb

[NbConvertApp] Converting notebook 14_Mapping_Actores_del_Conflicto_NNAJ.ipynb to html
[NbConvertApp] Writing 700331 bytes to 14_Mapping_Actores_del_Conflicto_NNAJ.html
